<a href="https://colab.research.google.com/github/juanpajaro/IA_en_salud_diplomado_puj/blob/main/clase_2_regression_ml_salud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install shap

In [ ]:
# imports
import pandas as pd
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import shap
from lightgbm import LGBMRegressor, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn import ensemble, linear_model

In [ ]:
# read in dataset
df = pd.read_csv("LengthOfStay.csv")
df.head()

In [ ]:
df.info()

#Limpieza de datos

https://microsoft.github.io/r-server-hospital-length-of-stay/input_data.html

In [ ]:
# type conversions
datetime_cols = ["vdate", "discharged"]
cat_cols = ["gender", "rcount", "facid"]
bin_cols = ["dialysisrenalendstage",
            "asthma",
            "irondef",
            "pneum",
            "substancedependence",
            "psychologicaldisordermajor",
            "depress",
            "psychother",
            "fibrosisandother",
            "malnutrition",
            "hemo"]
num_cols = ["hematocrit",
            "neutrophils",
            "sodium",
            "glucose",
            "bloodureanitro",
            "creatinine",
            "bmi",
            "respiration"]

for date_col, cat_col in zip(datetime_cols, cat_cols):
    df[date_col] = pd.to_datetime(df[date_col], format="%m/%d/%Y")
    df[cat_col] = df[cat_col].astype("category")
df.head()

In [ ]:
df.info()

#Ingeniería de caracteristicas

In [ ]:
# number_of_issues transform function
def calculate_number_of_issues(df, bin_cols):
    df["numberofissues"] = df[bin_cols].sum(axis=1)
    return df

df = calculate_number_of_issues(df, bin_cols)

#Cargue del algoritmo

In [ ]:
labels, features = df[["lengthofstay"]], df.drop(["lengthofstay", "discharged", "vdate", "eid"], axis=1)
labels.value_counts()

In [ ]:
features.info()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels)

In [ ]:
regressor = make_pipeline(
    ColumnTransformer([
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(), cat_cols),
    ], remainder="passthrough"),
    #Agregar su algoritmo
    ensemble.GradientBoostingRegressor()
    #fin
)

#Ajusto el algoritmo para obtener un modelo

In [ ]:
regressor.fit(x_train, y_train)

#Predicciones y evaluacion

In [ ]:
preds = np.round(regressor.predict(x_test), 0)
r2_score(preds, y_test)

In [ ]:
explainer = shap.Explainer(regressor[1], feature_names=regressor[:-1].get_feature_names_out())

observations = regressor[0].transform(x_test)
shap_values = explainer(observations)
shap_values.shape

In [ ]:
shap.plots.beeswarm(shap_values,  max_display=30)
plt.savefig('beeswarm_lengthofstay.jpg')